<a href="https://colab.research.google.com/github/Samgomes2510/dashborad-ecommerce/blob/main/3_PreparacaoDadosModelagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, MinMaxScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
spark = SparkSession.builder.getOrCreate()

In [15]:
pagamentos_normalizado = spark.read.option('header', 'true').parquet('drive/MyDrive/Colab Notbooks/spark/pagamentos_normalizado.parquet')
join_itens_pedido = spark.read.option('header', 'true').parquet('drive/MyDrive/Colab Notebooks/spark/join_itens_pedido_parquet')

In [17]:
pagamentos_normalizado.show(n=5, truncate=False)
join_itens_pedido.show(n=5, truncate=False)

+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|id_pedido                       |sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|valor_pagamento_normalizado|parcelas_pagamento_normalizado|
+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|b81ef226f3fe1789b1e8b2acac839d17|1                  |credit_card   |8                 |99.33          |0.007269424652080491       |0.3333333333333333            |
|a9810da82917af2d9aefd1278f1dcfa0|1                  |credit_card   |1                 |24.39          |0.0017849719849415402      |0.041666666666666664          |
|25e8ea4e93396b6fa0d3dd708e76c1bd|1                  |credit_card   |1                 |65.71          |0.004808958963940492       |0.041666666666666664          |
|ba78997921bbcdc

In [18]:
pagamentos_normalizado.corr('valor_pagamento_normalizado', 'parcelas_pagamento_normalizado')

0.33081084451898507

In [19]:
itens_pedido_df = join_itens_pedido.na.drop()

montar_vetor = VectorAssembler(
    inputCols=['peso_produto_g', 'comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm'],
    outputCol='caracteristicas'
    )

itens_pedido_df = montar_vetor.transform(itens_pedido_df)
itens_pedido_df.show()

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     caracteristicas|
+--------------------+--------------------+--------------+--------------------+---------------

In [22]:
Scaler = MinMaxScaler(inputCol='caracteristicas', outputCol='caracteristicas_normalizado')
modelo_scaler = Scaler.fit(itens_pedido_df)
itens_pedido_df = modelo_scaler.transform(itens_pedido_df)

In [23]:
train_data, test_data = itens_pedido_df.randomSplit([0.7, 0.3], seed=42)

# Visualizar os tamanahos dos conjuntos
print(f"treino: {train_data.count()},teste: {test_data.count()}")

treino: 76178,teste: 32465


In [25]:
# Modelo de regressão linear
regressao_linear = LinearRegression(
    featuresCol='caracteristicas_normalizado',
    labelCol='valor_frete')


modelo_lr = regressao_linear.fit(train_data)

avaliar_test = modelo_lr.evaluate(test_data)
print("RMSE(Erro Quadrático Médio da Raiz):", avaliar_test.rootMeanSquaredError)
print("r2 (Coeficiente de Determinação):", avaliar_test.r2)

RMSE(Erro Quadrático Médio da Raiz): 12.29261332362075
r2 (Coeficiente de Determinação): 0.3656767551944773


In [26]:
spark.stop()